Evan Callia, Kyle Kusuda

This scraper takes the top 10 places to visit in Seattle. It then goes into the link for those places and grabs the address. Once the address has been obtained it uses nominatum to convert the addresses to point coordinates and exports these points into geoJSON and shapefiles

If we wanted to grab information from a certain website we could decide what to grab and use it
as freely as the website would let us. In this case we took addresses and used the Nominatim API
to get the lat-long coordinates to map.

Some difficulties we had along the way were converting to geoJson files with the correct spatial reference
and aligning ourselves in terms of hierarchy within the loops/lists/dictionaries.
    
Evan and Kyle collaborated on the code as well as looking at online code sources for ideas on how to solve
the more complext problems.
    
Time spent on code: 7-10 hours each

In [13]:
#setting system pathways for arcmap
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')

#setting up ability to export as geojson file
from subprocess import call
import os
os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"

#importhing methods used
import urllib2
import lxml.html
import requests
import json
import arcpy
arcpy.env.workspace = r"V:\UW\Geog 458\Lab03"
arcpy.env.overwriteOutput = True

#Set up html parser
url = "http://www.tripadvisor.com/Attractions-g60878-Activities-Seattle_Washington.html"
html = urllib2.urlopen(url).read()
dom = lxml.html.fromstring(html)
dom.make_links_absolute(url)

#grab all places mentioned on website
places = dom.cssselect("div.property_title > a:first-child")

#make a list of place names
titles = [link.text for link in places]

#make a list of all the links
links = [link.attrib['href'] for link in places]

#list of all the places (populated later)
allPlaces = []

#go into links and pull out addresses for top 3 places
i = 0
while(i < 3):
    #set up html parser for second page
    newurl = links[i]
    htmltwo = urllib2.urlopen(newurl).read()
    domtwo = lxml.html.fromstring(htmltwo)
    domtwo.make_links_absolute(newurl)
        
    #grab all components of the address
    street = domtwo.cssselect("span.street-address")
    city = domtwo.cssselect("span[property=addressLocality]")
    state = domtwo.cssselect("span[property=addressRegion]")
    zipcode = domtwo.cssselect("span[property=postalCode]")

    #creates the actual text names from span objects
    streets = [strName.text for strName in street]
    cities = [ctName.text for ctName in city] #Is this necessary or can we just put Seattle?
    states = [stName.text for stName in state] #or just WA?
    zipcodes = [zipC.text for zipC in zipcode]
        
    #make dictionary out of title/street/city/state/zip/link and populate our allPlaces list with them
    allPlaces.append({"Title": titles[i], "Street": streets, "City": cities, "State": states, "Zipcode": zipcodes, "Link": links[i]})
    i += 1

#make feature class to put our points in (setting refrecences and adding fields needed)
sr = arcpy.SpatialReference(4326)
arcpy.CreateFeatureclass_management(r"V:\UW\Geog 458\Lab03", "test.shp", "POINT","", "", "", sr)
arcpy.AddField_management(r"V:\UW\Geog 458\Lab03\test.shp", "NAME", "Text")

#list of our points to be made (populated later)
points = []

count = 0
for row in allPlaces:
    #concatinate url searches and retrieve data in json format
    query = "http://nominatim.openstreetmap.org/search?q=" + row["Street"][0].replace(" ", "+") + "+" + row["City"][0].replace(" ", "+") + ",+" + row["State"][0].replace(" ", "+") + "+" + row["Zipcode"][0].replace(" ", "+") + "&format=json&polygon=1&addressdetails=1"    
    response = requests.get(query)
    response_list = json.loads(response.content)
        
    #grab latitude and longitude from json returned
    lat = ""
    lon = ""
    for row2 in response_list:
        if "lat" in row2:
            lat = row2["lat"]
        if "lon" in row2:
            lon = row2["lon"]
            break
            
    #put the name and coordinates of each point in our points list
    points.append((titles[count], [float(lon), float(lat)]))
    count += 1

#populate shapefile with out points
cursor = arcpy.da.InsertCursor(r"V:\UW\Geog 458\Lab03\test.shp", ("NAME", "SHAPE@XY"))
for row in points:
    cursor.insertRow(row)
del cursor
   
#convert shapefile to geoJSON file
call(['C:\\OSGeo4W\\bin\\ogr2ogr','-f','GeoJSON','-t_srs','WGS84','-s_srs','EPSG:4326', 'V:\\UW\\Geog 458\\Lab03\\test.geojson','V:\\UW\\Geog 458\\Lab03\\test.shp'])

    

    
    

0